In [43]:
import pandas as pd
import nltk
from cleantext.clean import clean
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tqdm import tqdm
import swifter
import ast


In [43]:
tqdm.pandas()

In [80]:
df = pd.read_csv('data/news_sample.csv')

In [81]:
def clean_text_library(text):
    return clean(str(text),  # Convert to string in case of non-string input
        fix_unicode=True,               
        to_ascii=True,                  
        lower=True,                     
        no_line_breaks=True,            
        lang="en"                       
    )
df['content'] = df['content'].astype(str).progress_apply(clean_text_library)


# Save cleaned data
output_path = 'data/news_sample_1.csv'
df.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")


Pandas Apply: 100%|██████████| 250/250 [00:00<00:00, 554.72it/s]

Cleaned data saved to data/news_sample_1.csv


In [ ]:
# Regex patterns 
patterns = {
    r"[\w]+ [\d]+, [\d]+": "",                    # DATE method 1
    r"[\d]+[\w]+ [\w]+ [\d]+": "",                # DATE method 2
    r"[\d]+\/?-?\.?[\d]+\/?-?\.?[\d]+": "",       # DATE method 3 (fixed brackets)
    r"[\w]+ \d\d?[\w]?[\w]?,? [\d]{2,4}": "",     # DATE method 4
    r"([\d]{1,2}[\w]*) ([\w]*),? ([\d]{2,4})": "",# DATE method 5 (fixed range)
    r"\b(\d+(st|nd|rd|th|s))\b": ""                # NUM
}

def clean_dates(df):
    for pattern, replacement in patterns.items():
        df['content'] = df['content'].str.replace(pattern, replacement, regex=True)
    return df

df = clean_dates(df)
df.to_csv('data/news_sample_2.csv', index=False)

In [83]:
def clean_text_library(text):
    return clean(text,
    no_urls=True,
    lower=False,                   
    no_emails=True,               
    no_phone_numbers=True,        
    no_numbers=True,               
    no_digits=True,                
    no_currency_symbols=True,      
    no_punct=True,                
    replace_with_punct="",          
    replace_with_url="URL",
    replace_with_email="EMAIL",
    replace_with_phone_number="",
    replace_with_number="NUM",
    replace_with_digit="0",
    replace_with_currency_symbol="",
    lang="en"                       
)
# Cleaning the content column of the dataset
df['content'] = df['content'].progress_apply(clean_text_library)
df.to_csv('data/news_sample_3.csv', index=False)



Pandas Apply: 100%|██████████| 250/250 [00:00<00:00, 565.60it/s]


In [84]:
# Cleaning the content column of the dataset
df['content'] = df['content'].apply(word_tokenize)
df.to_csv('data/news_sample_4.csv', index=False)



In [16]:
stop_words = set(stopwords.words('english'))

def rs(list):
    return [word for word in list if word.lower() not in stop_words]

df['content'] = df['content'].apply(rs)
df.to_csv('data/news_sample_5.csv', index=False)


In [50]:
stemmer = PorterStemmer()
def setmming(list):
    return [stemmer.stem(word) for word in list]

df['content'] = df['content'].apply(setmming)
df.to_csv('data/news_sample_6.csv', index=False)


In [51]:
# Defines the regex for finding words  
word_pattern = re.compile(r"\b\w+\b")

# Load the cleaned dataset
df_5 = pd.read_csv('data/news_sample_5.csv')
df_6 = pd.read_csv('data/news_sample_6.csv')
# Extracting all words of the "news_sample.csv" and the cleaned version
df_5_content = " ".join(df_5["content"].astype(str))
df_6_content = " ".join(df_6["content"].astype(str))

df_5_content = word_pattern.findall(df_5_content)
df_6_content = word_pattern.findall(df_6_content)

# Extracting all unique words of the "news_sample.csv" and the cleaned version
len_df_5_content = len(set(df_5_content))
len_df_6_content = len(set(df_6_content))

# Printing the unique words of the raw version, and the unique words after it has been cleaned 
print(f"Number of unique words in the df_5_content text: {len_df_5_content}")
print(f"Number of unique words in the df_6_content text: {len_df_6_content}")

print(len_df_6_content/len_df_5_content)

Number of unique words in the df_5_content text: 16371
Number of unique words in the df_6_content text: 10958
0.6693543460998106


In [6]:
df_995 = pd.read_csv('data/995,000_rows.csv')

/var/folders/z1/hf1tn63x7tl1qjxy1md9xy4w0000gn/T/ipykernel_45185/100071585.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_995 = pd.read_csv('data/995,000_rows.csv')


In [ ]:
def clean_space(txt):
    return clean(str(txt), fix_unicode=True, to_ascii=True, lower=True, no_line_breaks=True,lang="en")

# Clean the text remove newline tabs spaces
df_995['content'] = df_995['content'].astype(str).swifter.progress_bar(True).apply(clean_space)

# Save cleaned data
output_path = 'data/995,000_row_cleaned_spaces.csv'
df_995.to_csv(output_path, index=False)
print(f"Cleaned the newline tabs spaces; data saved to {output_path}")



Pandas Apply: 100%|██████████| 995000/995000 [16:53<00:00, 981.86it/s] 


Cleaned the newline tabs spaces; data saved to data/995,000_row_cleaned_spaces.csv


In [ ]:
patterns = {
    r"[\w]+ [\d]+, [\d]+": "",                     # DATE method 1
    r"[\d]+[\w]+ [\w]+ [\d]+": "",                 # DATE method 2
    r"[\d]+\/?-?\.?[\d]+\/?-?\.?[\d]+": "",        # DATE method 3 (fixed brackets)
    r"[\w]+ \d\d?[\w]?[\w]?,? [\d]{2,4}": "",      # DATE method 4
    r"([\d]{1,2}[\w]*) ([\w]*),? ([\d]{2,4})": "", # DATE method 5 (fixed range)
    r"\b(\d+(st|nd|rd|th|s))\b": ""                # NUM
}
compiled_patterns = [(re.compile(pattern), replacement) for pattern, replacement in patterns.items()]

def remove_dates(txt):
    for pattern, replacement in compiled_patterns:
        txt = re.sub(pattern, replacement, txt)
    return txt

# Remove dates
df_995['content'] = df_995['content'].astype(str).swifter.progress_bar(True).apply(remove_dates)

# Save cleaned data
output_path = 'data/995,000_row_remove_dates.csv'
df_995.to_csv(output_path, index=False)
print(f"Remove dates; data saved to {output_path}")


Pandas Apply: 100%|██████████| 995000/995000 [04:58<00:00, 3337.23it/s]


Remove dates; data saved to data/995,000_row_remove_dates.csv


In [98]:
def remove_urls(txt):
    return clean(txt, no_urls=True, no_emails=True, no_phone_numbers=True, no_numbers=True, no_digits=True, no_currency_symbols=True, lower=False, no_punct=True, replace_with_punct="", replace_with_url="URL", replace_with_email="EMAIL", replace_with_phone_number="", replace_with_number="NUM", replace_with_digit="0", replace_with_currency_symbol="", lang="en")

# Remove urls, emails, phone numbers, numbers, digits, currency symbols, punctuations
df_995['content'] = df_995['content'].astype(str).swifter.progress_bar(True).apply(remove_urls)

# Save cleaned data
output_path = 'data/995,000_row_removed_urls.csv'
print(f"Removed urls, emails, phone numbers, numbers, digits, currency symbols, punctuations; data saved to {output_path}")
df_995.to_csv(output_path, index=False)

Pandas Apply: 100%|██████████| 995000/995000 [18:59<00:00, 873.44it/s] 


Removed urls, emails, phone numbers, numbers, digits, currency symbols, punctuations; data saved to data/995,000_row_removed_urls.csv


In [99]:

# Tokenize the text
df_995['content'] = df_995['content'].astype(str).swifter.progress_bar(True).apply(word_tokenize)

# Save cleaned data
output_path = 'data/995,000_row_tokenized.csv'
df_995.to_csv(output_path, index=False)
print(f"Tokenized; data saved to {output_path}")

Pandas Apply: 100%|██████████| 995000/995000 [09:33<00:00, 1735.19it/s] 


Tokenized; data saved to data/995,000_row_tokenized.csv


In [42]:
df_995 = pd.read_csv('data/995,000_row_tokenized.csv')

/var/folders/z1/hf1tn63x7tl1qjxy1md9xy4w0000gn/T/ipykernel_45185/3962938543.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_995 = pd.read_csv('data/995,000_row_tokenized.csv')


In [44]:

stop_words = set(stopwords.words('english'))
def remove_stop_words(lst):
    return [word for word in lst if word.lower() not in stop_words]

# Remove stop words
df_995['content'] = df_995['content'].swifter.progress_bar(True).apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else x)

# Save cleaned data
output_path = 'data/995,000_row_removed_stop_words.csv'
df_995.to_csv(output_path, index=False)
print(f"Row_removed_stop_words; data saved to {output_path}")


Pandas Apply: 100%|██████████| 995000/995000 [12:53<00:00, 1286.54it/s] 


Row_removed_stop_words; data saved to data/995,000_row_removed_stop_words.csv


In [47]:

stop_words = stopwords.words('english')
print(stop_words)
print(df_995['content'].head())

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [41]:
import ast

# Load NLTK stop words as a set for fast lookups
stop_words_set = set(stopwords.words('english'))

def remove_stop_words(word_list):
    """Remove stop words efficiently from a list of words using NLTK."""
    return [word for word in word_list if word.lower() not in stop_words_set]

# Load the dataset
df = pd.read_csv('data/news_sample_4.csv')

# Convert 'content' column from string to actual list if needed
df['content'] = df['content'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else x)

# Apply stop word removal with swifter for efficiency
df['content'] = df['content'].swifter.progress_bar(True).apply(remove_stop_words)

# Print results
print(df.head())
print(df['content'].head())

Pandas Apply: 100%|██████████| 250/250 [00:00<00:00, 32636.43it/s]

   Unnamed: 0   id                domain        type  \
0           0  141               awm.com  unreliable   
1           1  256     beforeitsnews.com        fake   
2           2  700           cnnnext.com  unreliable   
3           3  768               awm.com  unreliable   
4           4  791  bipartisanreport.com   clickbait   

                                                 url  \
0  http://awm.com/church-congregation-brings-gift...   
1  http://beforeitsnews.com/awakening-start-here/...   
2  http://www.cnnnext.com/video/18526/never-hike-...   
3  http://awm.com/elusive-alien-of-the-sea-caught...   
4  http://bipartisanreport.com/2018/01/21/trumps-...   

                                             content  \
0  [sometimes, power, christmas, make, wild, wond...   
1  [awakening, NUM, strands, dna, reconnecting, m...   
2  [never, hike, alone, friday, fan, film, usa, |...   
3  [rare, shark, caught, scientists, left, blunde...   
4  [donald, trump, unnerving, ability, ability

In [ ]:
stemmer = PorterStemmer()
def stemming(lst):
    return [stemmer.stem(word) for word in lst]

#  Stem the text
df_995['content'] = df_995['content'].swifter.progress_bar(True).apply(setmming)

# Save cleaned data
output_path = 'data/995,000_row_cleaned.csv'
df_995.to_csv(output_path, index=False)
print(f"Fully cleaned; data saved to {output_path}")

Pandas Apply:   1%|          | 12413/995000 [00:34<26:06, 627.13it/s]  